<a href="https://colab.research.google.com/github/DSabarish/LoanDataAnalytics/blob/main/bankloan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sqlalchemy_schemadisplay

In [3]:
import pandas as pd
from google.cloud import bigquery


In [6]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [17]:
# Setting up BigQuery client
client = bigquery.Client(project="bankloanproject")

# Query to see all the data inside the database
query = """

SELECT *
FROM T.bank
LIMIT 1000

"""

# Execute the query
query_job = client.query(query)
# Fetch results
results = query_job.result()
# Convert results into a DataFrame
df = pd.DataFrame(data=[row.values() for row in results], columns=[field.name for field in results.schema])
# Adding index to the DataFrame
df = df.reset_index(drop=True)
# Display the DataFrame
df.sample(4)

,id,address_state,application_type,emp_length,emp_title,grade,home_ownership,issue_date,last_credit_pull_date,last_payment_date,...,sub_grade,term,verification_status,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment
283,226154,MN,INDIVIDUAL,10+ years,Ramsey County,E,MORTGAGE,2021-01-08,2021-01-11,2021-02-11,...,E1,36 months,Not Verified,72000.0,0.2257,170.72,0.1393,5000,44,6146
546,260928,NY,INDIVIDUAL,< 1 year,St Jerome School,C,RENT,2021-02-08,2021-10-10,2021-11-10,...,C2,36 months,Not Verified,38000.0,0.0872,394.80,0.1134,12000,7,14178
362,360414,KY,INDIVIDUAL,5 years,Commonwealth of Kentucky,C,MORTGAGE,2021-11-08,2021-01-16,2021-11-11,...,C1,36 months,Verified,41004.0,0.1531,529.75,0.1178,16000,16,19071
174,308498,NJ,INDIVIDUAL,2 years,Emergency Medical Associate,B,MORTGAGE,2021-04-08,2021-03-11,2021-04-11,...,B3,36 months,Verified,225000.0,0.0451,587.14,0.1008,25000,39,21137


In [28]:
# Total Loan Applications

query = """

SELECT COUNT(id) AS Total_Applications FROM T.bank

"""
df = client.query(query).to_dataframe()
df


,Total_Applications
0,38576


In [31]:
# MTD Loan Applications

query = """
SELECT COUNT(id) AS Total_Applications
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 12
"""

df = client.query(query).to_dataframe()
df


,Total_Applications
0,4314


In [32]:
# PMTD Loan Applications

query = """
SELECT COUNT(id) AS Total_Applications
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 11
"""

df = client.query(query).to_dataframe()
df


,Total_Applications
0,4035


In [33]:
# Total Funded Amount

query = """
SELECT SUM(loan_amount) AS Total_Funded_Amount
FROM T.bank
"""

df = client.query(query).to_dataframe()
df


,Total_Funded_Amount
0,435757075


In [34]:
# MTD Total Funded Amount

query = """
SELECT SUM(loan_amount) AS Total_Funded_Amount
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 12
"""

df = client.query(query).to_dataframe()
df


,Total_Funded_Amount
0,53981425


In [35]:
# PMTD Total Funded Amount

query = """
SELECT SUM(loan_amount) AS Total_Funded_Amount
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 11
"""

df = client.query(query).to_dataframe()
df


,Total_Funded_Amount
0,47754825


In [36]:
# Total Amount Collected

query = """
SELECT SUM(total_payment) AS Total_Amount_Collected
FROM T.bank
"""

df = client.query(query).to_dataframe()
df


,Total_Amount_Collected
0,473070933


In [37]:
# MTD Total Amount Received

query = """
SELECT SUM(total_payment) AS Total_Amount_Collected
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 12
"""

df = client.query(query).to_dataframe()
df


,Total_Amount_Collected
0,58074380


In [38]:
# PMTD Total Amount Received

query = """
SELECT SUM(total_payment) AS Total_Amount_Collected
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 11
"""

df = client.query(query).to_dataframe()
df

,Total_Amount_Collected
0,50132030


In [39]:
# Average Interest Rate

query = """
SELECT AVG(int_rate)*100 AS Avg_Int_Rate
FROM T.bank
"""

df = client.query(query).to_dataframe()
df


,Avg_Int_Rate
0,12.048831


In [40]:
# MTD Average Interest Rate

query = """
SELECT AVG(int_rate)*100 AS MTD_Avg_Int_Rate
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 12
"""

df = client.query(query).to_dataframe()
df


,MTD_Avg_Int_Rate
0,12.356041


In [41]:
# PMTD Average Interest Rate

query = """
SELECT AVG(int_rate)*100 AS PMTD_Avg_Int_Rate
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 11
"""

df = client.query(query).to_dataframe()
df


,PMTD_Avg_Int_Rate
0,11.941717


In [42]:
# Avg DTI

query = """
SELECT AVG(dti)*100 AS Avg_DTI
FROM T.bank
"""

df = client.query(query).to_dataframe()
df


,Avg_DTI
0,13.327433


In [43]:
# MTD Avg DTI

query = """
SELECT AVG(dti)*100 AS MTD_Avg_DTI
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 12
"""

df = client.query(query).to_dataframe()
df


,MTD_Avg_DTI
0,13.665538


In [44]:
# PMTD Avg DTI

query = """
SELECT AVG(dti)*100 AS PMTD_Avg_DTI
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 11
"""

df = client.query(query).to_dataframe()
df


,PMTD_Avg_DTI
0,13.302734


In [45]:
# Good Loan Percentage

query = """
SELECT
    (COUNT(CASE WHEN loan_status = 'Fully Paid' OR loan_status = 'Current' THEN id END) * 100.0) /
    COUNT(id) AS Good_Loan_Percentage
FROM T.bank
"""

df = client.query(query).to_dataframe()
df


,Good_Loan_Percentage
0,86.175342


In [46]:
# Good Loan Applications

query1 = """
SELECT COUNT(id) AS Good_Loan_Applications
FROM T.bank
WHERE loan_status = 'Fully Paid' OR loan_status = 'Current'
"""

df1 = client.query(query1).to_dataframe()
df1


,Good_Loan_Applications
0,33243


In [47]:
# Good Loan Funded Amount

query2 = """
SELECT SUM(loan_amount) AS Good_Loan_Funded_amount
FROM T.bank
WHERE loan_status = 'Fully Paid' OR loan_status = 'Current'
"""

df2 = client.query(query2).to_dataframe()
df2


,Good_Loan_Funded_amount
0,370224850


In [48]:
# Good Loan Amount Received

query1 = """
SELECT SUM(total_payment) AS Good_Loan_amount_received
FROM T.bank
WHERE loan_status = 'Fully Paid' OR loan_status = 'Current'
"""

df1 = client.query(query1).to_dataframe()
df1


,Good_Loan_amount_received
0,435786170


In [49]:
# Bad Loan Percentage

query2 = """
SELECT
    (COUNT(CASE WHEN loan_status = 'Charged Off' THEN id END) * 100.0) /
    COUNT(id) AS Bad_Loan_Percentage
FROM T.bank
"""

df2 = client.query(query2).to_dataframe()
df2


,Bad_Loan_Percentage
0,13.824658


In [50]:
# Bad Loan Applications

query3 = """
SELECT COUNT(id) AS Bad_Loan_Applications
FROM T.bank
WHERE loan_status = 'Charged Off'
"""

df3 = client.query(query3).to_dataframe()
df3


,Bad_Loan_Applications
0,5333


In [51]:
# Bad Loan Funded Amount

query4 = """
SELECT SUM(loan_amount) AS Bad_Loan_Funded_amount
FROM T.bank
WHERE loan_status = 'Charged Off'
"""

df4 = client.query(query4).to_dataframe()
df4


,Bad_Loan_Funded_amount
0,65532225


In [52]:
# Bad Loan Amount Received

query5 = """
SELECT SUM(total_payment) AS Bad_Loan_amount_received
FROM T.bank
WHERE loan_status = 'Charged Off'
"""

df5 = client.query(query5).to_dataframe()
df5


,Bad_Loan_amount_received
0,37284763


#LOAN STATUS

In [53]:
# LOAN STATUS

query1 = """
SELECT
    loan_status,
    COUNT(id) AS LoanCount,
    SUM(total_payment) AS Total_Amount_Received,
    SUM(loan_amount) AS Total_Funded_Amount,
    AVG(int_rate * 100) AS Interest_Rate,
    AVG(dti * 100) AS DTI
FROM
    T.bank
GROUP BY
    loan_status
"""

df1 = client.query(query1).to_dataframe()
df1


,loan_status,LoanCount,Total_Amount_Received,Total_Funded_Amount,Interest_Rate,DTI
0,Fully Paid,32145,411586256,351358350,11.641071,13.167351
1,Charged Off,5333,37284763,65532225,13.878575,14.004733
2,Current,1098,24199914,18866500,15.099326,14.724344


In [56]:
# MTD LOAN STATUS

query2 = """
SELECT
    loan_status,
    SUM(total_payment) AS MTD_Total_Amount_Received,
    SUM(loan_amount) AS MTD_Total_Funded_Amount
FROM T.bank
WHERE EXTRACT(MONTH FROM issue_date) = 12
GROUP BY loan_status
"""

df = client.query(query2).to_dataframe()
df


,loan_status,MTD_Total_Amount_Received,MTD_Total_Funded_Amount
0,Fully Paid,47815851,41302025
1,Charged Off,5324211,8732775
2,Current,4934318,3946625


In [57]:
# BANK LOAN REPORT | OVERVIEW

query = """
SELECT
    EXTRACT(MONTH FROM issue_date) AS Month_Number,
    FORMAT_DATE('%B', issue_date) AS Month_Name,
    COUNT(id) AS Total_Loan_Applications,
    SUM(loan_amount) AS Total_Funded_Amount,
    SUM(total_payment) AS Total_Amount_Received
FROM T.bank
GROUP BY Month_Number, Month_Name
ORDER BY Month_Number
"""

df = client.query(query).to_dataframe()
df


,Month_Number,Month_Name,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
0,1,January,2332,25031650,27578836
1,2,February,2279,24647825,27717745
2,3,March,2627,28875700,32264400
3,4,April,2755,29800800,32495533
4,5,May,2911,31738350,33750523
5,6,June,3184,34161475,36164533
6,7,July,3366,35813900,38827220
7,8,August,3441,38149600,42682218
8,9,September,3536,40907725,43983948
9,10,October,3796,44893800,49399567


In [58]:
# STATE LOAN REPORT

query = """
SELECT
    address_state AS State,
    COUNT(id) AS Total_Loan_Applications,
    SUM(loan_amount) AS Total_Funded_Amount,
    SUM(total_payment) AS Total_Amount_Received
FROM T.bank
GROUP BY State
ORDER BY State
"""

df = client.query(query).to_dataframe()
df


,State,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
0,AK,78,1031800,1108570
1,AL,432,4949225,5492272
2,AR,236,2529700,2777875
3,AZ,833,9206000,10041986
4,CA,6894,78484125,83901234
5,CO,770,8976000,9845810
6,CT,730,8435575,9357612
7,DC,214,2652350,2921854
8,DE,110,1138100,1269136
9,FL,2773,30046125,31601905


In [59]:
# TERM LOAN REPORT

query1 = """
SELECT
    term AS Term,
    COUNT(id) AS Total_Loan_Applications,
    SUM(loan_amount) AS Total_Funded_Amount,
    SUM(total_payment) AS Total_Amount_Received
FROM T.bank
GROUP BY Term
ORDER BY Term
"""

df1 = client.query(query1).to_dataframe()
df1


,Term,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
0,36 months,28237,273041225,294709458
1,60 months,10339,162715850,178361475


In [60]:
# EMPLOYEE LENGTH LOAN REPORT

query2 = """
SELECT
    emp_length AS Employee_Length,
    COUNT(id) AS Total_Loan_Applications,
    SUM(loan_amount) AS Total_Funded_Amount,
    SUM(total_payment) AS Total_Amount_Received
FROM T.bank
GROUP BY Employee_Length
ORDER BY Employee_Length
"""

df2 = client.query(query2).to_dataframe()
df2


,Employee_Length,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
0,1 year,3229,32883125,35498348
1,10+ years,8870,116115950,125871616
2,2 years,4382,44967975,49206961
3,3 years,4088,43937850,47551832
4,4 years,3428,37600375,40964850
5,5 years,3273,36973625,40397571
6,6 years,2228,25612650,27908658
7,7 years,1772,20811725,22584136
8,8 years,1476,17558950,19025777
9,9 years,1255,15084225,16516173


In [61]:
# PURPOSE LOAN REPORT

query3 = """
SELECT
    purpose AS Purpose,
    COUNT(id) AS Total_Loan_Applications,
    SUM(loan_amount) AS Total_Funded_Amount,
    SUM(total_payment) AS Total_Amount_Received
FROM T.bank
GROUP BY Purpose
ORDER BY Purpose
"""

df3 = client.query(query3).to_dataframe()
df3


,Purpose,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
0,Debt consolidation,18214,232459675,253801871
1,car,1497,10223575,11324914
2,credit card,4998,58885175,65214084
3,educational,315,2161650,2248380
4,home improvement,2876,33350775,36380930
5,house,366,4824925,5185538
6,major purchase,2110,17251600,18676927
7,medical,667,5533225,5851372
8,moving,559,3748125,3999899
9,other,3824,31155750,33289676


In [62]:
# HOME OWNERSHIP LOAN REPORT

query4 = """
SELECT
    home_ownership AS Home_Ownership,
    COUNT(id) AS Total_Loan_Applications,
    SUM(loan_amount) AS Total_Funded_Amount,
    SUM(total_payment) AS Total_Amount_Received
FROM T.bank
GROUP BY Home_Ownership
ORDER BY Home_Ownership
"""

df4 = client.query(query4).to_dataframe()
df4


,Home_Ownership,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
0,MORTGAGE,17198,219329150,238474438
1,NONE,3,16800,19053
2,OTHER,98,1044975,1025257
3,OWN,2838,29597675,31729129
4,RENT,18439,185768475,201823056


In [63]:
# Filter by Grade A for PURPOSE LOAN REPORT

query = """
SELECT
    purpose AS Purpose,
    COUNT(id) AS Total_Loan_Applications,
    SUM(loan_amount) AS Total_Funded_Amount,
    SUM(total_payment) AS Total_Amount_Received
FROM T.bank
WHERE grade = 'A'
GROUP BY Purpose
ORDER BY Purpose
"""

df = client.query(query).to_dataframe()
df


,Purpose,Total_Loan_Applications,Total_Funded_Amount,Total_Amount_Received
0,Debt consolidation,3753,37216300,38822971
1,car,577,3629475,3805538
2,credit card,1353,12636075,13339495
3,educational,79,484000,515639
4,home improvement,933,8359175,8744006
5,house,91,916575,957878
6,major purchase,796,5344575,5604259
7,medical,197,1471850,1526882
8,moving,164,1069450,1099875
9,other,1024,7043175,7397982
